In [7]:
search_for = 120
start_from = 133819999
threads = 5
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004483699798583984
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
50000 0.19664441769202287
100000 0.18811489943662488
150000 0.09810645184220076
200000 0.1122906329817878
250000 0.17985236303513713
300000 0.19400796160049438
350000 0.19343174243460762
400000 0.1787966368711167
450000 0.10012387596616745
500000 0.1627767600755996
550000 0.1789336654234091
600000 0.19910228583987158
650000 0.20004341668061418
700000 0.16862494889330995
750000 0.16723158703383206
800000 0.1801448709594263
850000 0.2018497059492363
900000 0.19231417136254308
950000 0.19001894082895918
1000000 0.11741385574168763
1050000 0.1296145592090885
1100000 0.19589522902938528
1150000 0.20112638809578154
1200000 0.18516786254568102
1250000 0.16050390891229843
1300000 0.163269903993641
1350000 0.1653623173187415
1400000 0.14646993815710543
1450000 0.179213

14650000 0.1900029370920115
14700000 0.15770976638674736
14750000 0.1784636229620258
14800000 0.14546055697566004
14850000 0.19386323681220613
14900000 0.18971184128834007
14950000 0.15505287774832116
15000000 0.09774029626322189
15050000 0.1677186587930613
15100000 0.19811857732854551
15150000 0.20051436458330155
15200000 0.19314832434821264
15250000 0.19264959852266972
15300000 0.19711584995352427
15350000 0.19763325810365542
15400000 0.19349814817556404
15450000 0.19388445820364159
15500000 0.1904948616748876
15550000 0.16653457213071585
15600000 0.11894807004263401
15650000 0.10367069474807182
15700000 0.10199703234902488
15750000 0.1162441653112491
15800000 0.17144484707455104
15850000 0.09832359412857558
15900000 0.11086095370064973
15950000 0.10549462279467052
16000000 0.16080524851411182
16050000 0.18995524393794538
16100000 0.15986830630780458
16150000 0.195538700877383
16200000 0.196377750644509
16250000 0.11439065289112992
16300000 0.09831834518655008
16350000 0.112821541805

29100000 0.10078670934488773
29150000 0.18027980480491188
29200000 0.19045176403300498
29250000 0.18565327904307447
29300000 0.11676341028200123
29350000 0.18984247590882117
29400000 0.11251491470378636
29450000 0.11299049352564945
29500000 0.18896419560545946
29550000 0.18230470241324903
29600000 0.19394367021858558
29650000 0.11852327316514386
29700000 0.14371713794371685
29750000 0.15394462300948436
29800000 0.1343681703309099
29850000 0.19343573430764277
29900000 0.19359516951811842
29950000 0.19672555719600252
30000000 0.19246094767644406
30050000 0.16556190845402877
30100000 0.15480849197781882
30150000 0.1861770652136644
30200000 0.10174194187715715
30250000 0.10044276217760377
30300000 0.13283354643984238
30350000 0.19773751132937276
30400000 0.17379074252684248
30450000 0.09687080874917507
30500000 0.17128700138431788
30550000 0.1912404663815326
30600000 0.19048540668839614
30650000 0.19023652071386044
30700000 0.19093943544052178
30750000 0.17935167994747164
30800000 0.126586

43500000 0.16001936045660972
43550000 0.09565792993829118
43600000 0.10133179017041524
43650000 0.09720312516580025
43700000 0.13535072838732004
43750000 0.18756713908901215
43800000 0.1013995379090031
43850000 0.09776448692224157
43900000 0.11058192693627014
43950000 0.17181074646070002
44000000 0.09474158626404604
44050000 0.09592148907373745
44100000 0.13202718469057878
44150000 0.1799985967629976
44200000 0.1745862499235352
44250000 0.14416916479385297
44300000 0.09252871055485673
44350000 0.09410020788288646
44400000 0.11376275825622083
44450000 0.18482001488856342
44500000 0.18908394639624357
44550000 0.19618909201056559
44600000 0.18861353889940846
44650000 0.18674977223782147
44700000 0.18973554521338543
44750000 0.11137695986855428
44800000 0.17667529440207214
44850000 0.18808565864528418
44900000 0.19199594675530463
44950000 0.17449105743140644
45000000 0.18692712833558317
45050000 0.1528321222828163
45100000 0.13320210343430175
45150000 0.16356652731487753
45200000 0.1904640

57750000 0.18434884017730954
57800000 0.11423006975405217
57850000 0.09823607852045695
57900000 0.11303763982053995
57950000 0.1909998184424758
58000000 0.1870902192594555
58050000 0.18793349424631597
58100000 0.1913959789204346
58150000 0.19373143148141966
58200000 0.15300029640757243
58250000 0.09298212964538335
58300000 0.08985591825262572
58350000 0.09617627381082773
58400000 0.11247845590075121
58450000 0.09381354237727722
58500000 0.14777354517184896
58550000 0.1825158792206844
58600000 0.18198335981352598
58650000 0.18895729959701296
58700000 0.19482547045570875
58750000 0.1917742020668917
58800000 0.19052568057966232
58850000 0.15814676264390148
58900000 0.13826929555231599
58950000 0.19134304091529447
59000000 0.15581391822342872
59050000 0.16596206200324032
59100000 0.18812083032034635
59150000 0.18796072669252156
59200000 0.18419988266918794
59250000 0.18360740389494895
59300000 0.1844512263853219
59350000 0.1841639429768271
59400000 0.19036857558959724
59450000 0.1919047833

...

1101750000 0.322653521562771
1101800000 0.4553462096988347
1101850000 0.2319140360298157
1101900000 0.4032685233712673
best so far: 0
type: [2, 4, 1, 3, 5] 120
cases of this type: 67500
1101950000 0.39837726702409587
1102000000 0.23954160145032805
best so far: 0
type: [2, 4, 1, 5, 3] 120
cases of this type: 40500
best so far: 0
type: [2, 4, 1, 15, 1] 120
cases of this type: 13500
1102050000 0.13190350375543036
best so far: 0
type: [2, 4, 3, 1, 5] 120
cases of this type: 22500
best so far: 0
type: [2, 4, 3, 5, 1] 120
cases of this type: 4500
best so far: 0
type: [2, 4, 5, 1, 3] 120
cases of this type: 8100
best so far: 0
type: [2, 4, 5, 3, 1] 120
cases of this type: 2700
best so far: 0
type: [2, 4, 15, 1, 1] 120
cases of this type: 900
best so far: 0
type: [2, 5, 1, 1, 12] 120
cases of this type: 103680
1102100000 0.12470628518348138
1102150000 0.17524063852208988
1102200000 0.16542656619215407
best so far: 0
type: [2, 5, 1, 2, 6] 120
cases of this type: 51840
1102250000 0.1325492037523

1108000000 0.24786723195949129
1108050000 0.2270252424426635
1108100000 0.24446281519010066
1108150000 0.24054311576770016
best so far: 0
type: [3, 1, 1, 20, 2] 120
cases of this type: 128000
1108200000 0.16227024866588116
1108250000 0.10383801639727884
1108300000 0.10503949954178597
best so far: 0
type: [3, 1, 1, 40, 1] 120
cases of this type: 64000
1108350000 0.10149511700152952
best so far: 0
type: [3, 1, 2, 1, 20] 120
cases of this type: 640000
1108400000 0.2932349220651507
1108450000 0.2070198238833507
1108500000 0.23561142522838513
1108550000 0.2587252129254791
1108600000 0.22661327931415373
1108650000 0.13125933436813356
1108700000 0.11211318884796169
1108750000 0.11032058727836742
1108800000 0.19892396422007086
1108850000 0.3150708808813042
1108900000 0.26402282144836453
1108950000 0.189753639915359
1109000000 0.25075472847741975
best so far: 0
type: [3, 1, 2, 2, 10] 120
cases of this type: 320000
1109050000 0.21142589950271448
1109100000 0.19321908745101693
1109150000 0.194070

1112850000 0.06327706550003688
1112900000 0.06497131019351747
1112950000 0.06416437115977208
best so far: 0
type: [4, 1, 1, 6, 5] 120
cases of this type: 135000
1113000000 0.06250207723915975
1113050000 0.04883663221111165
best so far: 0
type: [4, 1, 1, 10, 3] 120
cases of this type: 81000
1113100000 0.059708812172893685
1113150000 0.0503031915802598
best so far: 0
type: [4, 1, 1, 15, 2] 120
cases of this type: 54000
1113200000 0.05100092857479387
best so far: 0
type: [4, 1, 1, 30, 1] 120
cases of this type: 27000
1113250000 0.03819411381768385
best so far: 0
type: [4, 1, 2, 1, 15] 120
cases of this type: 202500
1113300000 0.059269108385602634
1113350000 0.03510333865244918
1113400000 0.028070066148796344
1113450000 0.06979578729973236
best so far: 0
type: [4, 1, 2, 3, 5] 120
cases of this type: 67500
1113500000 0.03740333529784415
best so far: 0
type: [4, 1, 2, 5, 3] 120
cases of this type: 40500
1113550000 0.028859479868140483
best so far: 0
type: [4, 1, 2, 15, 1] 120
cases of this t

best so far: 0
type: [5, 2, 3, 1, 4] 120
cases of this type: 4608
1115400000 0.008835062383818627
best so far: 0
type: [5, 2, 3, 2, 2] 120
cases of this type: 2304
best so far: 0
type: [5, 2, 3, 4, 1] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 4, 1, 3] 120
cases of this type: 2592
best so far: 0
type: [5, 2, 4, 3, 1] 120
cases of this type: 864
best so far: 0
type: [5, 2, 6, 1, 2] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 6, 2, 1] 120
cases of this type: 576
best so far: 0
type: [5, 2, 12, 1, 1] 120
cases of this type: 288
best so far: 0
type: [5, 3, 1, 1, 8] 120
cases of this type: 12288
best so far: 0
type: [5, 3, 1, 2, 4] 120
cases of this type: 6144
best so far: 0
type: [5, 3, 1, 4, 2] 120
cases of this type: 3072
best so far: 0
type: [5, 3, 1, 8, 1] 120
cases of this type: 1536
best so far: 0
type: [5, 3, 2, 1, 4] 120
cases of this type: 3072
best so far: 0
type: [5, 3, 2, 2, 2] 120
cases of this type: 1536
best so far: 0
type: [5, 3, 2, 4, 1] 120
case

best so far: 0
type: [10, 3, 1, 2, 2] 120
cases of this type: 384
best so far: 0
type: [10, 3, 1, 4, 1] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 1, 2] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 2, 1] 120
cases of this type: 96
best so far: 0
type: [10, 3, 4, 1, 1] 120
cases of this type: 48
best so far: 0
type: [10, 4, 1, 1, 3] 120
cases of this type: 324
best so far: 0
type: [10, 4, 1, 3, 1] 120
cases of this type: 108
best so far: 0
type: [10, 4, 3, 1, 1] 120
cases of this type: 36
best so far: 0
type: [10, 6, 1, 1, 2] 120
cases of this type: 96
best so far: 0
type: [10, 6, 1, 2, 1] 120
cases of this type: 48
best so far: 0
type: [10, 6, 2, 1, 1] 120
cases of this type: 24
best so far: 0
type: [10, 12, 1, 1, 1] 120
cases of this type: 12
best so far: 0
type: [12, 1, 1, 1, 10] 120
cases of this type: 10000
best so far: 0
type: [12, 1, 1, 2, 5] 120
cases of this type: 5000
best so far: 0
type: [12, 1, 1, 5, 2] 120
cases of this type: 2000
best so far